In [1]:
import pyprob
from pyprob import Model
from pyprob.distributions import Normal, Uniform, Categorical, Exponential

Cannot import dbm.gnu: No module named '_gdbm'


In [4]:
# number of leaves
class MarkovChainTree(Model):
    def __init__(self, depth=5):
        super().__init__(name="Markov Chain Path") # give the model a name
        self.depth = depth

    def forward(self): # Needed to specify how the generative model is run forward
        # sample the (latent) mean variable to be inferred:
        paths = [[0,],]  # up to 2^d-1 long list of the full paths for each 'particle'
        moves = {0: -1, 1: 1}
        for i in range(self.depth):
            for j in range(len(paths)):
                coord = paths[j][-1]
                #splitting = pyprob.sample(Normal(0,1), name = 'split{}{}'.format(i,j))
                #if splitting.item() < -1:
                splitting = pyprob.sample(Exponential(1.5), name = 'split{}{}'.format(i,j))
                if splitting < 0.2:
                    move = pyprob.sample(Categorical([1/2.,1/2.]), name = 'input_split{}{}'.format(i,j))
                    move = moves[move.item()]
                    new_path = paths[j].copy()
                    new_path.append(coord + move)
                    paths.append(new_path)
                move = pyprob.sample(Categorical([1/2.,1/2.]), name = 'input{}{}'.format(i,j))
                move = moves[move.item()]
                paths[j].append(coord + move)

        obs_distr = Normal(len(paths), 0.3)
        pyprob.observe(obs_distr, name='obs0') # NOTE: observe -> denotes observable variables
        return paths

model = MarkovChainTree(6)

In [5]:
model.forward()

[[0, 1, 0, -1, -2, -3, -2],
 [0, 1, 0, -1, 0, -1, 0],
 [0, 1, 0, -1, -2, -3, -2],
 [0, 1, 0, 1, 2, 3, 2],
 [0, 1, 0, -1, -2, -1, -2],
 [0, 1, 0, -1, -2, -3, -2],
 [0, 1, 0, -1, 0, -1, -2],
 [0, 1, 0, -1, -2, -3, -2],
 [0, 1, 0, 1, 2, 3, 4]]

In [6]:
model.learn_inference_network(
    num_traces=10000,
    observe_embeddings={'obs0': {'dim': 100, 'depth': 5}}
)

Creating new inference network...
Observable obs0: reshape not specified, using shape torch.Size([]).
Observable obs0: using embedding dim torch.Size([100]).
Observable obs0: observe embedding not specified, using the default FEEDFORWARD.
Observable obs0: using embedding depth 5.
Observe embedding dimension: 100
Train. time | Epoch| Trace     | Init. loss| Min. loss | Curr. loss| T.since min | Learn.rate| Traces/sec
New layers, address: 86__forward__splitting__Exponential__1, distribution: Exponential


RuntimeError: Distribution currently unsupported: Exponential